## File for different Model Layers and the like

In [1]:
import torch
from torch import nn

In [3]:
class selfAttentionDot(nn.Module):
    """calculating dot product self attention
    """
    def __init__(self):
        super(selfAttentionDot,self).__init__()
    
    def forward(self,q,k,v):
        #q,k,v = (batch,seq,features)
        # add activations
        depth = q.shape[2]
        s = torch.matmul(k,q.permute([0,2,1]))/torch.sqrt(depth) #(batch,seq,seq)
        return torch.matmul(s,v) #(batch,seq,features)
        
        
        
        

In [66]:
seq = 10
batch = 8
features = 7

k = torch.rand(batch,seq,features)
q = torch.rand(batch,seq,features)
v = torch.rand(batch,seq,features)

s = torch.matmul(k,q.permute([0,2,1]))
print(s.shape)
torch.matmul(s,v).shape


torch.Size([8, 10, 10])


torch.Size([8, 10, 7])

In [46]:
test.mean([-1,-2])
len((1,2,3))

3

In [37]:
# test = torch.tensor([[1,1,1],[2,2,2],[3,3,3],[4,4,4]]).float()
# mean = torch.mean(test, dim = 1,keepdim=True)
# print(mean)
# # print(test.shape,mean.shape)
# mean = mean.repeat(1,3)
# # print(test.shape,mean.shape)
# # torch.sub(test,mean,)
# print(test)
# print(mean)
# print(test-mean)

In [38]:
class LayerNorm(nn.Module):
    """My implementation of LayerNorm for 1d inputs from https://arxiv.org/pdf/1607.06450.pdf
    still have to figure out how this links up with a (batch_dim,channel,seq_length) input for transformers
    as well as fix some shapes
    """
    def __init__(self, input_shape):
        super(LayerNorm,self).__init__()
        
        self.bias = nn.parameter.Parameter(data=torch.zeros(input_shape),requires_grad=True)
        self.gain = nn.parameter.Parameter(data=torch.ones(input_shape),requires_grad=True)
    
        self.input_shape = input_shape
        self.dim = [-i for i in range(1,len(input_shape)+1)]
    def forward(self,inputs):
        """
        Args:
            inputs (torch.Tensor): tensor of shape(batch,seq,features))
        """
        batch_dim = inputs.shape[0]
        mu = torch.mean(inputs,dim=self.dim)
        var = torch.mean(torch.square(inputs - mu.repeat(1,self.input_dim)),dim = 1)
        std = torch.sqrt(var)
        
        centered = (inputs - mu.repeat(1,self.inpu_dim))
        
        gain = self.gain.repeat(batch_dim, 1)
        bias = self.bias.repeat(batch_dim, 1)
        
        scale = std/gain
        
        norm = scale*centered + bias
        return norm
        
        
        

In [48]:
t = torch.rand(12,8,20)
torch.flatten(t,0,1).shape

torch.Size([96, 20])

In [ ]:
class TransformerEncoder(nn.Module):
    #(batch, seq, features)
    def __init__(self,size):
        super(TransformerEncoder,self).__init__()
        
        self.keys = nn.Linear(size, size)
        self.query = nn.Linear(size, size)
        self.values = nn.Linear(size, size)
        
        self.selfAttention = selfAttentionDot()
        
    def forward(self,inputs):
        #(batch, seq, features)
        shape = inputs.shape
        flatten = torch.flatten(inputs,0,1)
        k = self.keys(flatten)
        k = torch.reshape(k,shape)
        
        q = self.query(flatten)
        q = torch.reshape(q,shape)
        
        v = self.values(flatten)
        v = torch.reshape(v,shape)
        
        attention = self.selfAttention(k,q,v)
        
        return inputs + attention
                
        
        
        